In [1]:
import pandas as pd
import matplotlib.pyplot as plt 

from bs4 import BeautifulSoup
import requests
import time
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

In [2]:
df_movies = pd.read_csv('movies.csv') 

In [3]:
df_ratings = pd.read_csv('ratings.csv')

In [4]:
df_links = pd.read_csv('links.csv', dtype = object)
df_links.fillna(value=0, inplace=True)

In [5]:
df_tags = pd.read_csv('tags.csv')

In [ ]:
df_movies

In [ ]:
df_ratings

In [ ]:
df_links

In [ ]:
df_tags

In [6]:
header = {"accept-language": "es-ES"}

In [32]:
def sacar_sinopsis(df_links, header):
    results = []
    for i, tmdbId in enumerate(df_links["tmdbId"]):
        link = "https://www.themoviedb.org/movie/" + str(tmdbId)
        r = requests.get(link, headers = header)
        bs = BeautifulSoup(r.content, "html.parser")
        try:
            sinopsis = bs.p.get_text()
            movie_id = df_links.at[i,"movieId"]
            results.append([movie_id,tmdbId,sinopsis])
        except:
            results.append([tmdbId,"No se encontro sinopsis"])
            continue
        time.sleep(1)
        print(f"{i+1}/{len(df_links)} completed")

    df_sinopsis = pd.DataFrame(results,columns=['movieId','tmdbId','sinopsis'])
    df_sinopsis.to_csv('sinopsis.csv', index=False)
    return df_sinopsis
df_sinopsis = sacar_sinopsis(df_links, header)

1/9742 completed
2/9742 completed
3/9742 completed


KeyboardInterrupt: 

In [45]:
df_sinopsis = pd.read_csv('sinopsis.csv')

In [46]:
df_usuario = pd.read_csv('Usuario_0.csv')

In [47]:
def juntarMoviesSinopsis(df_movies, df_sinopsis, movieId, movies_sinopsis):
    # Unir los dataframes en función de la columna 'movieId'
    df_movies_sinopsis = pd.merge(df_movies, df_sinopsis, on='movieId')

    # Eliminar columna tmdbId
    df_movies_sinopsis = df_movies_sinopsis.drop('tmdbId', axis=1)
    
    # Guardar el resultado en un nuevo archivo CSV
    df_movies_sinopsis.to_csv('movies_sinopsis.csv', index=False)    
    return df_movies_sinopsis

In [48]:
#Comprobar que crea y elimina y guarda en un fichero csv
df_movies_sinopsis = juntarMoviesSinopsis(df_movies, df_sinopsis, 'movieId', 'movies_sinopsis.csv')
print(df_movies_sinopsis)

      movieId                                      title  \
0           1                           Toy Story (1995)   
1           2                             Jumanji (1995)   
2           3                    Grumpier Old Men (1995)   
3           4                   Waiting to Exhale (1995)   
4           5         Father of the Bride Part II (1995)   
...       ...                                        ...   
9737   193581  Black Butler: Book of the Atlantic (2017)   
9738   193583               No Game No Life: Zero (2017)   
9739   193585                               Flint (2017)   
9740   193587        Bungo Stray Dogs: Dead Apple (2018)   
9741   193609        Andrew Dice Clay: Dice Rules (1991)   

                                           genres  \
0     Adventure|Animation|Children|Comedy|Fantasy   
1                      Adventure|Children|Fantasy   
2                                  Comedy|Romance   
3                            Comedy|Drama|Romance   
4             

In [49]:
def get_sinopsis(movie_id):
    # Leer el archivo CSV
    df = pd.read_csv("movies_sinopsis.csv")

    # Buscar la sinopsis en el archivo CSV
    for i, row in df.iterrows():
        if row["movieId"] == movie_id:
            return row["sinopsis"]
    return "Sinopsis no encontrada"

# Ejemplo de uso
movie_id = 2
sinopsis = get_sinopsis(movie_id)
print(sinopsis)

Alan Parris queda atrapado en un juego de mesa mágico y muy antiguo, Jumanji, durante 25 años. Cuando finalmente es liberado por dos niños, una manada de animales exóticos también es liberada. Alan deberá salvar a su pueblo de la destrucción.


In [52]:
def get_sinopsis(title):
    # Leer el archivo CSV
    df = pd.read_csv("movies_sinopsis.csv")

    # Buscar la sinopsis en el archivo CSV
    for i, row in df.iterrows():
        if row["title"] == title:
            return row["sinopsis"]
    return "Sinopsis no encontrada"

# Ejemplo de uso
title = "Toy Story (1995)"
sinopsis = get_sinopsis(title)
print(sinopsis)

Los juguetes de Andy, un niño de seis años, temen que haya llegado su hora y que un nuevo regalo de cumpleaños les sustituya en el corazón de su dueño. Woody, un vaquero que ha sido hasta ahora el juguete favorito de Andy, trata de tranquilizarlos hasta que aparece Buzz Lightyear, un héroe espacial dotado de todo tipo de avances tecnológicos. Woody es relegado a un segundo plano., pero su constante rivalidad se transformará en una gran amistad cuando ambos se pierden en la ciudad sin saber cómo volver a casa.


In [23]:
df_movies = pd.read_csv('movies.csv')
df_usuario = pd.read_csv('Usuario_0.csv')

In [24]:
def sacarPelisNoVistasPorU0(df_movies, df_usuario, movieId, PNVU0):
    # Unir los dataframes en función de la columna 'movieId'
    merged_df = pd.merge(df_movies, df_usuario, on='movieId', how='outer', indicator=True)
    
    dfPNVU0 = merged_df[merged_df['_merge'].isin(['left_only', 'right_only'])]
    # Eliminar columna tmdbId
    dfPNVU0.drop(columns='_merge', inplace=True)
    dfPNVU0.drop('title_y', axis=1, inplace=True)
    dfPNVU0 = dfPNVU0.rename(columns={'title_x': 'title'})
    
    # Guardar el resultado en un nuevo archivo CSV
    dfPNVU0.to_csv("PNVU0.csv", index=False)    
    return dfPNVU0

In [25]:
#Comprobar que crea y elimina y guarda en un fichero csv
dfPNVU0 = sacarPelisNoVistasPorU0(df_movies, df_usuario, 'movieId', 'PNVU0.csv')
print(dfPNVU0)

      movieId                                      title  \
2           3                    Grumpier Old Men (1995)   
3           4                   Waiting to Exhale (1995)   
4           5         Father of the Bride Part II (1995)   
6           7                             Sabrina (1995)   
7           8                        Tom and Huck (1995)   
...       ...                                        ...   
9738   193581  Black Butler: Book of the Atlantic (2017)   
9739   193583               No Game No Life: Zero (2017)   
9740   193585                               Flint (2017)   
9741   193587        Bungo Stray Dogs: Dead Apple (2018)   
9742   193609        Andrew Dice Clay: Dice Rules (1991)   

                               genres  rating  
2                      Comedy|Romance     NaN  
3                Comedy|Drama|Romance     NaN  
4                              Comedy     NaN  
6                      Comedy|Romance     NaN  
7                  Adventure|Children  

E:\Anaconda-Python\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [26]:
dfPNVU0

,movieId,title,genres,rating
2,3,Grumpier Old Men (1995),Comedy|Romance,NaN
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,NaN
4,5,Father of the Bride Part II (1995),Comedy,NaN
6,7,Sabrina (1995),Comedy|Romance,NaN
7,8,Tom and Huck (1995),Adventure|Children,NaN
...,...,...,...,...
9738,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,NaN
9739,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,NaN
9740,193585,Flint (2017),Drama,NaN
9741,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,NaN


In [ ]:

# Tokenizar
tokens = word_tokenize(sinopsis)

# Eliminar stopwords
stop_words = set(stopwords.words("spanish"))
tokens = [token for token in tokens if token.lower() not in stop_words]

# Stemming
stemmer = SnowballStemmer("spanish")
tokens = [stemmer.stem(token) for token in tokens]

# Frecuencia de palabras
freq = nltk.FreqDist(tokens)
print(freq.most_common(10))

In [54]:
class procesarTexto():
    
    def tokenize(text):
        
        return nltk.word_tokenize(text)
    
    def clean_text(text):
        # Eliminar números
        text = re.sub(r'\d+', '', text)
        # Eliminar signos de puntuación
        text = re.sub(r'[^\w\s]', '', text)
        # Convertir a minúsculas
        text = text.lower()
        return text

    def remove_stopwords(text):
        # Cargar lista de stopwords en español
        stop_words = set(stopwords.words("spanish"))
        # Tokenizar el texto
        tokens = word_tokenize(text)
        # Eliminar stopwords
        filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
        # Unir los tokens en una cadena de texto
        filtered_text = ' '.join(filtered_tokens)
        return filtered_text
    
    def stem_text(text):
        # Tokenizar el texto
        tokens = word_tokenize(text)
        # Crear un objeto stemmer
        stemmer = SnowballStemmer("spanish")
        # Aplicar stemming a cada token
        stemmed_tokens = [stemmer.stem(token) for token in tokens]
        # Unir los tokens en una cadena de texto
        stemmed_text = ' '.join(stemmed_tokens)
        return stemmed_text
    
    def word_frequency(text):
        # Tokenizar el texto
        tokens = word_tokenize(text)
        # Contar la frecuencia de cada token
        freq = nltk.FreqDist(tokens)
        # Devolver diccionario con la frecuencia de cada palabra
        return dict(freq)
    
    

In [ ]:
def get_sinopsis(title):
    # Leer el archivo CSV
    df = pd.read_csv("movies_sinopsis.csv")

    # Buscar la sinopsis en el archivo CSV
    for i, row in df.iterrows():
        if row["title"] == title:
            sinopsis = row["sinopsis"]
            sinopsis = clean_text(sinopsis)
            sinopsis = remove_stopwords(sinopsis)
            sinopsis = stem_text(sinopsis)
            frequency = word_frequency(sinopsis)
            return sinopsis, frequency
    return "Sinopsis no encontrada"

# Ejemplo de uso
title = "Toy Story (1995)"
sinopsis, frequency = get_sinopsis(title)
print(sinopsis)
print(frequency)